In [1]:
#Python
from __future__ import print_function

from pyrosetta import *

#Core Includes
from pyrosetta.rosetta.core.kinematics import MoveMap
from pyrosetta.rosetta.core.kinematics import FoldTree
from pyrosetta.rosetta.core.pack.task import TaskFactory
from pyrosetta.rosetta.core.pack.task import operation
from pyrosetta.rosetta.core.simple_metrics import metrics
from pyrosetta.rosetta.core.select import residue_selector as selections
from pyrosetta.rosetta.core.select.movemap import *

#Protocol Includes
from pyrosetta.rosetta.protocols import minimization_packing as pack_min
from pyrosetta.rosetta.protocols import relax as rel
from pyrosetta.rosetta.protocols.antibody.residue_selector import CDRResidueSelector
from pyrosetta.rosetta.protocols.antibody import *
from pyrosetta.rosetta.protocols.loops import *
from pyrosetta.rosetta.protocols.relax import FastRelax



#Intitlialization (Setting relax rounds to 2 instead of default 5 for speed of demo)
init('-use_input_sc -input_ab_scheme AHo_Scheme -ignore_unrecognized_res \
     -ignore_zero_occupancy false -load_PDB_components false -relax:default_repeats 2')

#Note that typically, we would add these options: -ex1 and -ex2 in order to increase the amount of rotamers
# available for packing, but this will slow us down for the demo, so we are keeping this out. 

PyRosetta-4 2020 [Rosetta PyRosetta4.conda.mac.python37.Release 2020.02+release.22ef835b4a2647af94fcd6421a85720f07eddf12 2020-01-05T17:31:56] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.mac.python37.Release r242 2020.02+release.22ef835b4a2 22ef835b4a2647af94fcd6421a85720f07eddf12 http://www.pyrosetta.org 2020-01-05T17:31:56
core.init: {0} command: PyRosetta -use_input_sc -input_ab_scheme AHo_Scheme -ignore_unrecognized_res -ignore_zero_occupancy false -load_PDB_components false -relax:default_repeats 2 -database /Users/paul/anaconda3/envs/pyrosetta_env/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=-1527265273 seed_offset=0 real_seed=-1527265273 thread_index=0
basic.random.init_rand

Let's load in an antibody/antigen complex. Please open this structure in PyMol also.

In [2]:
#Import a pose
pose = pose_from_pdb("2r0l_1_1.pdb")
original_pose = pose.clone()

core.chemical.GlobalResidueTypeSet: {0} Finished initializing fa_standard residue type set.  Created 980 residue types
core.chemical.GlobalResidueTypeSet: {0} Total time to initialize 1.09395 seconds.
core.import_pose.import_pose: {0} File '2r0l_1_1.pdb' automatically determined to be of type PDB
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  OXT on residue ARG:CtermProteinFull 108
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  OXT on residue SER:CtermProteinFull 225
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  OXT on residue ARG:CtermProteinFull 464
core.conformation.Conformation: {0} Found disulfide between residues 23 88
core.conformation.Conformation: {0} current variant for 23 CYS
core.conformation.Conformation: {0} current variant for 88 CYS
core.conformation.Conformation: {0} current variant for 23 CYD
core.conformation.Conformation: {0} current variant for 88 CYD
core.conformation.Conformation: {0} Found disulfide

In [3]:
# PyMOL mover
pmm = pyrosetta.PyMOLMover()
pmm.apply(pose)

In [4]:
#Setup a normal TaskFactory
tf = TaskFactory()
tf.push_back(operation.InitializeFromCommandline())
tf.push_back(operation.RestrictToRepacking())

In [5]:
#Setup The packer
packer = pack_min.PackRotamersMover()
packer.task_factory(tf)

#Note that we are not passing a scorefunction here.  We will use the default, cmd-line scorefunction, 
# which is access through rosetta.core.scoring.get_score_function().  We use use a scorefunction later. 

#Run the packer once; this can take a few seconds
packer.apply(pose)

#Dump the PDB
pose.dump_pdb('2r0l_all_repack.pdb')

protocols.minimization_packing.PackRotamersMover: {0} [ WARNING ] undefined ScoreFunction -- creating a default one
core.scoring.ScoreFunctionFactory: {0} SCOREFUNCTION: ref2015
core.scoring.etable: {0} Starting energy table calculation
core.scoring.etable: {0} smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: {0} smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: {0} smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: {0} Finished calculating energy tables.
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/HBEval.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/DonStrength.csv
basic.io.database: {

True

In [6]:
#1
#Lets compare the energies of the before and after packing. Any difference?
scorefxn = get_score_function()
before = scorefxn.score(original_pose)
#Finish the code here
# YOUR CODE HERE
after = scorefxn.score(pose)
print('Difference in energies =', after - before)

core.scoring.ScoreFunctionFactory: {0} SCOREFUNCTION: ref2015
Difference in energies = -513.0752291652267


Now let's look at the sequences of the pose and the original pose. You can get a structure's sequence using `Pose`'s sequence function. 

Make sure that two the poses have the same sequence. When we used the "restrict to repacking" residue-level task operation, we should have prevented the packer from changing any amino acids.

In [7]:
#2
# YOUR CODE HERE
print(pose.sequence())
print()
print(original_pose.sequence())

DIQMTQSPSSLSASVGDRVTITCRASQDVSTAVAWYQQKPGKAPKLLIYSASFLYSGVPSRFSGSGSGTDFTLTISSLQPEDFATYYCQQSYTTPPTFGQGTKVEIKREVQLVESGGGLVQPGGSLRLSCAASGFTISNSGIHWVRQAPGKGLEWVGWIYPTGGATDYADSVKGRFTISADTSKNTAYLQMNSLRAEDTAVYYCARFWWRSFDYWGQGTLVTVSSIIGGSSSLPGSHPWLAAIYIGDSFCAGSLVHTCWVVSAAHCFSHSPPRDSVSVVLGQHFFNRTTDVTQTFGIEKYIPYTLYSVFNPSDHDLVLIRLKKKGDRCATRSQFVQPICLPEPGSTFPAGHKCQIAGWGHLDENVSGYSSSLREALVPLVADHKCSSPEVYGADISPNMLCAGYFDCKSDACQGDSGGPLACEKNGVAYLYGIISWGDGCGRLHKPGVYTRVANYVDWINDRIR

DIQMTQSPSSLSASVGDRVTITCRASQDVSTAVAWYQQKPGKAPKLLIYSASFLYSGVPSRFSGSGSGTDFTLTISSLQPEDFATYYCQQSYTTPPTFGQGTKVEIKREVQLVESGGGLVQPGGSLRLSCAASGFTISNSGIHWVRQAPGKGLEWVGWIYPTGGATDYADSVKGRFTISADTSKNTAYLQMNSLRAEDTAVYYCARFWWRSFDYWGQGTLVTVSSIIGGSSSLPGSHPWLAAIYIGDSFCAGSLVHTCWVVSAAHCFSHSPPRDSVSVVLGQHFFNRTTDVTQTFGIEKYIPYTLYSVFNPSDHDLVLIRLKKKGDRCATRSQFVQPICLPEPGSTFPAGHKCQIAGWGHLDENVSGYSSSLREALVPLVADHKCSSPEVYGADISPNMLCAGYFDCKSDACQGDSGGPLACEKNGVAYLYGIISWGDGCGRLHKPGVYTRVANYVDWINDRIR


Lets pack just a single CDR loop.  How do we do this?  Use a TaskFactory and an operation.  
To make things easier, we will use a Residue Selector to select the CDR. You will see more of this later.

In [8]:
nbr_selector = selections.NeighborhoodResidueSelector()
cdr_selector = CDRResidueSelector()
cdr_selector.set_cdr(h1)

nbr_selector.set_focus_selector(cdr_selector)
nbr_selector.set_include_focus_in_subset(True)

The cdr selector is pretty useful. Let's get the list of residues that are part of the H1 CDR so that we can look at where they are in PyMol.

In [9]:
cdr_selection = cdr_selector.apply(pose)
cdr_res = [res for res in range(1,len(cdr_selection)+1) if cdr_selection[res]]
pdb_res = [pose.pdb_info().pose2pdb(res) for res in cdr_res]
print(pdb_res)
print("select h1, chain H and res %s" % ("+".join([x.split(" ")[0] for x in pdb_res])))
print("show sticks, h1 and not elem h")
print("color yellow, h1")

basic.io.database: {0} Database file opened: sampling/antibodies/cluster_center_dihedrals.txt
protocols.antibody.AntibodyNumberingParser: {0} Antibody numbering scheme definitions read successfully
protocols.antibody.AntibodyNumberingParser: {0} Antibody CDR definition read successfully
antibody.AntibodyInfo: {0} Successfully finished the CDR definition
antibody.AntibodyInfo: {0} AC Detecting Regular CDR H3 Stem Type
antibody.AntibodyInfo: {0} ARFWWRSFDYW
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: KINKED
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: Kink: 1 Extended: 0
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H1
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 13 Omega: TTTTTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H2
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 10 Omega: TTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H3
protocols.antibody.cluster.CDRC

Copy the commands that were just printed out in last three lines of the cell above and paste them into PyMol. Take a look at where H1 is. Kinda at the interface, but perhaps not at the dead center of the interface. We're going to redesign this loop to see if we can't get a better sequence. As we go through this lab, you may very well find these three commands useful to re-run. You can press the up arrow button when you're in PyMol to scroll through your command history just like you can do on the command line.

---

Lets turn off packing for everything but the H1 loop and its neighbors. By using a TaskOperation and putting it in a TaskFactory, the PackRotamersMover will regenerate the list of neighbors of H1 every time its apply method is called. (The alternative would be to create a PackerTask ahead of time with the list of residues that neighbor H1 at the start. That list might not remain accurate if the conformation of H1 changes.)

We'll use the NeighborhoodResidueSelector to identify the neighbors of the H1 residues. This class is smart enough to use the neighbor relationships held in the pose's energies object, instead of re-calculating the neighbors (which is decently time consuming).

In [10]:
prevent_repacking_rlt = operation.PreventRepackingRLT()

#Setting "flip_subset" to true means that we're inverting the
# selection that the nbr_selector returns. We are turning off
# everything (with the prevent_repacking_rlt) except the CDR
# residues and their neighbors.
prevent_subset_repacking = operation.OperateOnResidueSubset(
    prevent_repacking_rlt, nbr_selector, flip_subset=True )

#Reset the pose
pose.assign(original_pose)

tf.push_back(prevent_subset_repacking)

pack_cdrs_and_neighbors_tf = tf.clone()

packer.task_factory(tf)

In [11]:
#3
#Run the packer
# YOUR CODE HERE
#Run the packer once; this can take a few seconds
packer.apply(pose)

# Note how many rotamers were used and how many positions were done?
# Now run it again. Have those numbers changed? Why or why not?
#core.pack.pack_rotamers: {0} built 827 rotamers at 60 positions.

core.pack.task: {0} Packer task: initialize from command line()
basic.io.database: {0} Database file opened: sampling/antibodies/cluster_center_dihedrals.txt
protocols.antibody.AntibodyNumberingParser: {0} Antibody numbering scheme definitions read successfully
protocols.antibody.AntibodyNumberingParser: {0} Antibody CDR definition read successfully
antibody.AntibodyInfo: {0} Successfully finished the CDR definition
antibody.AntibodyInfo: {0} AC Detecting Regular CDR H3 Stem Type
antibody.AntibodyInfo: {0} ARFWWRSFDYW
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: KINKED
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: Kink: 1 Extended: 0
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H1
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 13 Omega: TTTTTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H2
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 10 Omega: TTTTTTTTTT
antibody.AntibodyInfo: {

In [12]:
#Run the packer once; this can take a few seconds
packer.apply(pose)
# core.pack.pack_rotamers: {0} built 827 rotamers at 60 positions.

core.pack.task: {0} Packer task: initialize from command line()
basic.io.database: {0} Database file opened: sampling/antibodies/cluster_center_dihedrals.txt
protocols.antibody.AntibodyNumberingParser: {0} Antibody numbering scheme definitions read successfully
protocols.antibody.AntibodyNumberingParser: {0} Antibody CDR definition read successfully
antibody.AntibodyInfo: {0} Successfully finished the CDR definition
antibody.AntibodyInfo: {0} AC Detecting Regular CDR H3 Stem Type
antibody.AntibodyInfo: {0} ARFWWRSFDYW
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: KINKED
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: Kink: 1 Extended: 0
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H1
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 13 Omega: TTTTTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H2
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 10 Omega: TTTTTTTTTT
antibody.AntibodyInfo: {

OK. Lets design the H1 CDR. We have almost everything we need created, so lets do this!!

In [13]:
#4
pose.assign(original_pose)

tf.clear()
tf.push_back(operation.InitializeFromCommandline())
tf.push_back(prevent_subset_repacking)
# Note: by not appending the restrict-to-repacking task operation,
# we're effectively saying "design everything that can be packed"

#Set the new tf
# YOUR CODE HERE

#raise NotImplementedError()

#Run the packer
# YOUR CODE HERE
packer.apply(pose)

#Note this will take a significant amount of time (perhaps 5 or 10 minutes)
# core.pack.pack_rotamers: {0} built 12314 rotamers at 60 positions.

core.pack.task: {0} Packer task: initialize from command line()
basic.io.database: {0} Database file opened: sampling/antibodies/cluster_center_dihedrals.txt
protocols.antibody.AntibodyNumberingParser: {0} Antibody numbering scheme definitions read successfully
protocols.antibody.AntibodyNumberingParser: {0} Antibody CDR definition read successfully
antibody.AntibodyInfo: {0} Successfully finished the CDR definition
antibody.AntibodyInfo: {0} AC Detecting Regular CDR H3 Stem Type
antibody.AntibodyInfo: {0} ARFWWRSFDYW
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: KINKED
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: Kink: 1 Extended: 0
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H1
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 13 Omega: TTTTTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H2
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 10 Omega: TTTTTTTTTT
antibody.AntibodyInfo: {

In [14]:
#5
#Dump the PDB and take a look. 
# YOUR CODE HERE
pose.dump_pdb('2r0l_all_repack_step_5.pdb')

True

Okay, so we've now repacked the whole pose, repacked a CDR, and even designed a CDR.
But, we have a problem.  When we designed the CDR, we also designed its neighbors -- including the antigen. OOPS! One thing that you can't expect to do if you're designing an antibody for a particular pathogen is to change the sequence of the pathogen's proteins. "Oh yes, we can treat malaria, but not the malaria found in the wild; only the malaria we grew in the lab."  That won't fly at NIH.

We need to fix our code to confine our mutations to the H1 residues.

In order to tell the packer exactly what we want, we must approach the PackerTask-creation process as two steps now:

* Turn off packing everywhere but our CDR and its neighbors,
* Turn off design for the CDR neighbors

(These can be performed in any order)

Afterwards, the only place we'll be packing will be the CDR and its neighbors, and the place we'll be designing will be the CDR.

In [15]:
pose.assign(original_pose)
only_nbrs = selections.NeighborhoodResidueSelector()
only_nbrs.set_focus_selector(cdr_selector)
only_nbrs.set_include_focus_in_subset(False)

prevent_design_rlt = operation.RestrictToRepackingRLT()
prevent_design_on_neighbors = operation.OperateOnResidueSubset(prevent_design_rlt, only_nbrs)

tf.clear()
tf.push_back(operation.InitializeFromCommandline())
tf.push_back(prevent_subset_repacking)
tf.push_back(prevent_design_on_neighbors)

In [16]:
#6
#Set our new task factory to the PackRotamerMover and use its apply method

# YOUR CODE HERE
#Setup The packer
packer = pack_min.PackRotamersMover()
packer.task_factory(tf)

#Run the packer once; this can take a few seconds
packer.apply(pose)

protocols.minimization_packing.PackRotamersMover: {0} [ WARNING ] undefined ScoreFunction -- creating a default one
core.scoring.ScoreFunctionFactory: {0} SCOREFUNCTION: ref2015
core.pack.task: {0} Packer task: initialize from command line()
basic.io.database: {0} Database file opened: sampling/antibodies/cluster_center_dihedrals.txt
protocols.antibody.AntibodyNumberingParser: {0} Antibody numbering scheme definitions read successfully
protocols.antibody.AntibodyNumberingParser: {0} Antibody CDR definition read successfully
antibody.AntibodyInfo: {0} Successfully finished the CDR definition
antibody.AntibodyInfo: {0} AC Detecting Regular CDR H3 Stem Type
antibody.AntibodyInfo: {0} ARFWWRSFDYW
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: KINKED
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: Kink: 1 Extended: 0
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H1
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 13 Ome

Did the number of rotamers built change? Did the packer run faster than when we also redesigned the neighbors of H1? 

Let's use the SequenceMetric to take a look at the sequence (metrics.SequenceMetric). Use the constructor that takes a residue selector, and pass it the selector that identifies the H1 residues. Then you can get the sequence for those residues on a particular pose using the metric's `calculate` method.
```
   seqmet = metrics.SequenceMetric(<selector name>)
   seq1 = seqmet.calculate(pose1)
```

How does the sequence look? Did we get back the same sequence we started with? If not, do the mutations make sense? 

In [17]:
#7
# core.pack.pack_rotamers: {0} built 2944 rotamers at 60 positions.
# the packer seems to run faster
# YOUR CODE HERE
seqmet = metrics.SequenceMetric(cdr_selector)
seq1 = seqmet.calculate(pose)
print(seqmet)
print(seq1)

basic.io.database: {0} Database file opened: sampling/antibodies/cluster_center_dihedrals.txt
protocols.antibody.AntibodyNumberingParser: {0} Antibody numbering scheme definitions read successfully
protocols.antibody.AntibodyNumberingParser: {0} Antibody CDR definition read successfully
antibody.AntibodyInfo: {0} Successfully finished the CDR definition
antibody.AntibodyInfo: {0} AC Detecting Regular CDR H3 Stem Type
antibody.AntibodyInfo: {0} ARFWWRSFDYW
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: KINKED
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: Kink: 1 Extended: 0
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H1
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 13 Omega: TTTTTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H2
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 10 Omega: TTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H3
protocols.antibody.cluster.CDRC

# Relax

Lets first relax the whole protein. This will take some time, so after you start the apply function, take a break and stretch!

In [18]:
pose.assign(original_pose)
fr = FastRelax()

protocols.relax.RelaxScriptManager: {0} Reading relax scripts list from database.
core.scoring.ScoreFunctionFactory: {0} SCOREFUNCTION: ref2015
protocols.relax.RelaxScriptManager: {0} Looking for MonomerRelax2019.txt
protocols.relax.RelaxScriptManager: {0} ================== Reading script file: /Users/paul/anaconda3/envs/pyrosetta_env/lib/python3.7/site-packages/pyrosetta/database/sampling/relax_scripts/MonomerRelax2019.txt ==================
protocols.relax.RelaxScriptManager: {0} repeat %%nrepeats%%
protocols.relax.RelaxScriptManager: {0} coord_cst_weight 1.0
protocols.relax.RelaxScriptManager: {0} scale:fa_rep 0.040
protocols.relax.RelaxScriptManager: {0} repack
protocols.relax.RelaxScriptManager: {0} scale:fa_rep 0.051
protocols.relax.RelaxScriptManager: {0} min 0.01
protocols.relax.RelaxScriptManager: {0} coord_cst_weight 0.5
protocols.relax.RelaxScriptManager: {0} scale:fa_rep 0.265
protocols.relax.RelaxScriptManager: {0} repack
protocols.relax.RelaxScriptManager: {0} scale:fa_r

Here, we have to set a scorefunction or we segfault.  

That's embarrassing. Error checking is important, and protocols should use a default scorefunction if one is not provided.

We will manage.

In [19]:
fr.set_scorefxn(scorefxn)

In its default configuration, the FastRelax mover takes a very long time on a Pose this large, so we are
going decrease the number of minimization cycles we use.
This is not generally recommended, but we just want everything to run fast at the moment.

In [20]:
fr.max_iter(200)

Now, call the fast relax mover's apply function, but let's also measure how long it takes to run
```
%time my_mover.apply(my_pose)
```
so that we can see just how long this protocol takes.

In [21]:
#8
# YOUR CODE HERE
%time fr.apply(pose)

protocols.relax.FastRelax: {0} CMD: repeat  -520.742  0  0  0.55
protocols.relax.FastRelax: {0} CMD: coord_cst_weight  -520.742  0  0  0.55
protocols.relax.FastRelax: {0} CMD: scale:fa_rep  -895.085  0  0  0.022
core.pack.task: {0} Packer task: initialize from command line()
core.pack.pack_rotamers: {0} built 7459 rotamers at 464 positions.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating DensePDInteractionGraph
basic.thread_manager.RosettaThreadManager: {7} Thread 7 completed 2084 of 16922 work units.
basic.thread_manager.RosettaThreadManager: {1} Thread 1 completed 2309 of 16922 work units.
basic.thread_manager.RosettaThreadManager: {6} Thread 6 completed 2054 of 16922 work units.
basic.thread_manager.RosettaThreadManager: {3} Thread 3 completed 2161 of 16922 work units.
basic.thread_manager.RosettaThreadManager: {0} Thread 0 completed 1998 of 16922 work units.
bas

basic.thread_manager.RosettaThreadManager: {1} Thread 1 completed 2149 of 17039 work units.
basic.thread_manager.RosettaThreadManager: {4} Thread 4 completed 2112 of 17039 work units.
basic.thread_manager.RosettaThreadManager: {7} Thread 7 completed 2180 of 17039 work units.
basic.thread_manager.RosettaThreadManager: {2} Thread 2 completed 2205 of 17039 work units.
basic.thread_manager.RosettaThreadManager: {3} Thread 3 completed 2073 of 17039 work units.
basic.thread_manager.RosettaThreadManager: {6} Thread 6 completed 2042 of 17039 work units.
basic.thread_manager.RosettaThreadManager: {0} Thread 0 completed 2293 of 17039 work units.
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 8 available threads (8 had been requested).
protocols.relax.FastRelax: {0} CMD: repack  -1827.53  0.607516  0.607516  0.022
protocols.relax.FastRelax: {0} CMD: scale:fa_rep  -1819.69  0.607516  0.607516  0.02805
core.optimization.Minimizer: {0} [ WARNING ] LBFGS MAX CYC

In [22]:
#9
#Dump the pdb and take a look.
# YOUR CODE HERE
pose.dump_pdb('2r0l_all_repack_step_9.pdb')

True

In [23]:
#10
#Check the energy difference here once again.  Note how low it is compared to simply packing
# YOUR CODE HERE (COPIED FROM #1)
scorefxn = get_score_function()
before = scorefxn.score(original_pose)
after = scorefxn.score(pose)
print('Difference in energies =', after - before)

core.scoring.ScoreFunctionFactory: {0} SCOREFUNCTION: ref2015
Difference in energies = -991.8855325356991


So that was fun, but it was expensive. Usually when trying to get some task completed with Rosetta, we need to run a protocol repeatedly. If one protocol takes 10 mintues and another protocol takes 20 minutes, then we can run the 10-minute protocol twice as many times. If they both give equivalent results, then the 10-minute protocol is more likely to give us a useful design. So how can we optimize this protocol?

It would be cheaper and probably do what we need if we only optimized regions of the protein that we care about. This can be tricky, so to start off, lets do the basic thing and see what happens. 

Let's construct a MoveMap that tells the minimizer to only optimize the CDR and its neighbors. The fewer DOFs (degrees of freedom) to minimize, the faster the minimizer will run. 

We will use a MoveMapFactory to construct the MoveMap we want. The MoveMap factory allows us to use ResidueSelectors. First what will happen is the fast relax mover will construct a MoveMap and set some things in that MoveMap. Then it will give that MoveMap to the MoveMapFactory we're going to create. The MoveMapFactory then has the chance to modify the MoveMap. When the fast relax mover initializes the MoveMap, it sets both backbone and chi dihedrals set to change. First our MoveMapFactory will need to turn them off. Then we'll turn back *on* backbone and chi dihedrals for the CDR residues. Let's also let the sidechains minimize of the neighboring residues.

In [24]:
pose.assign(original_pose)
mmf = MoveMapFactory()

#mm_enable and mm_disable are Enums (numbered variables) that come when we import the MMF
mmf.all_bb(False)
mmf.all_chi(False)
mmf.add_bb_action(mm_enable, cdr_selector)
mmf.add_chi_action(mm_enable, nbr_selector) # keep the backbone fixed but let the sc's minimize

mm  = mmf.create_movemap_from_pose(pose)

basic.io.database: {0} Database file opened: sampling/antibodies/cluster_center_dihedrals.txt
protocols.antibody.AntibodyNumberingParser: {0} Antibody numbering scheme definitions read successfully
protocols.antibody.AntibodyNumberingParser: {0} Antibody CDR definition read successfully
antibody.AntibodyInfo: {0} Successfully finished the CDR definition
antibody.AntibodyInfo: {0} AC Detecting Regular CDR H3 Stem Type
antibody.AntibodyInfo: {0} ARFWWRSFDYW
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: KINKED
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: Kink: 1 Extended: 0
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H1
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 13 Omega: TTTTTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H2
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 10 Omega: TTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H3
protocols.antibody.cluster.CDRC

Lets take a look at our MoveMap that the MoveMap factory creates. Pass the movemap to `print()`.  We should only have a few residues there.

In [25]:
#11
# YOUR CODE HERE
print(mm)


-------------------------------
  resnum     Type  TRUE/FALSE 
-------------------------------
 DEFAULT      BB     FALSE
 DEFAULT      SC     FALSE
 DEFAULT      NU     FALSE
 DEFAULT  BRANCH     FALSE
     096      SC      TRUE
     109      SC      TRUE
     110      SC      TRUE
     111      SC      TRUE
     112      SC      TRUE
     130      SC      TRUE
     131      SC      TRUE
     132      SC      TRUE
     133      SC      TRUE
     134      BB      TRUE
              SC      TRUE
     135      BB      TRUE
              SC      TRUE
     136      BB      TRUE
              SC      TRUE
     137      BB      TRUE
              SC      TRUE
     138      BB      TRUE
              SC      TRUE
     139      BB      TRUE
              SC      TRUE
     140      BB      TRUE
              SC      TRUE
     141      BB      TRUE
              SC      TRUE
     142      BB      TRUE
              SC      TRUE
     143      BB      TRUE
              SC      TRUE
     144     

We'll set this movemap in the fast relax mover and then launch the mover. Remember to put `%time` in front of your call to `apply` so that we can know just how long this mover takes to run.

In [26]:
#12
# create a new fast relax mover to make sure
# nothing from our previous run bleeds through
fr = FastRelax()

# When we set a move map factory in the FastRelax mover,
# it will construct a new MoveMap ahead of each minimization
# call, and thus update the definition of the CDR neighborhood
# as the CDR moves. If the CDR were to move a lot, this could 
# make a big difference in the energies we produce 
fr.set_movemap_factory(mmf)
fr.set_scorefxn(scorefxn)
fr.max_iter(200)

#Here, we only want to pack the region and neighbors.  NOT all 500 residues!
fr.set_task_factory(pack_cdrs_and_neighbors_tf)

#Run this, take a break and then lets look at the results.
# YOUR CODE HERE
%time fr.apply(pose)

core.scoring.ScoreFunctionFactory: {0} SCOREFUNCTION: ref2015
basic.io.database: {0} Database file opened: sampling/antibodies/cluster_center_dihedrals.txt
protocols.antibody.AntibodyNumberingParser: {0} Antibody numbering scheme definitions read successfully
protocols.antibody.AntibodyNumberingParser: {0} Antibody CDR definition read successfully
antibody.AntibodyInfo: {0} Successfully finished the CDR definition
antibody.AntibodyInfo: {0} AC Detecting Regular CDR H3 Stem Type
antibody.AntibodyInfo: {0} ARFWWRSFDYW
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: KINKED
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: Kink: 1 Extended: 0
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H1
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 13 Omega: TTTTTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H2
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 10 Omega: TTTTTTTTTT
antibody.AntibodyInfo: {0}

antibody.AntibodyInfo: {0} Setting up CDR Cluster for L2
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 8 Omega: TTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for L3
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 9 Omega: TTTTTTCTT
core.pack.pack_rotamers: {0} built 880 rotamers at 60 positions.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating DensePDInteractionGraph
basic.thread_manager.RosettaThreadManager: {3} Thread 3 completed 145 of 1305 work units.
basic.thread_manager.RosettaThreadManager: {5} Thread 5 completed 134 of 1305 work units.
basic.thread_manager.RosettaThreadManager: {4} Thread 4 completed 150 of 1305 work units.
basic.thread_manager.RosettaThreadManager: {6} Thread 6 completed 227 of 1305 work units.
basic.thread_manager.RosettaThreadManager: {2} Thread 2 completed 190 of 1305 work units.
basic.thread_manager.Ros

protocols.relax.FastRelax: {0} CMD: repack  -589.076  0.0372573  0.0372573  0.55
protocols.relax.FastRelax: {0} CMD: min  -601.627  0.0513973  0.0513973  0.55
protocols.relax.FastRelax: {0} MRP: 0  -601.627  -601.627  0.0513973  0.0513973
protocols.relax.FastRelax: {0} CMD: accept_to_best  -601.627  0.0513973  0.0513973  0.55
protocols.relax.FastRelax: {0} CMD: endrepeat  -601.627  0.0513973  0.0513973  0.55
protocols.relax.FastRelax: {0} CMD: coord_cst_weight  -601.627  0.0513973  0.0513973  0.55
protocols.relax.FastRelax: {0} CMD: scale:fa_rep  -983.7  0.0513973  0.0513973  0.022
core.pack.task: {0} Packer task: initialize from command line()
basic.io.database: {0} Database file opened: sampling/antibodies/cluster_center_dihedrals.txt
protocols.antibody.AntibodyNumberingParser: {0} Antibody numbering scheme definitions read successfully
protocols.antibody.AntibodyNumberingParser: {0} Antibody CDR definition read successfully
antibody.AntibodyInfo: {0} Successfully finished the CDR de

protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 13 Omega: TTTTTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H2
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 10 Omega: TTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H3
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 10 Omega: TTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for L1
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 11 Omega: TTTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for L2
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 8 Omega: TTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for L3
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 9 Omega: TTTTTTCTT
core.pack.pack_rotamers: {0} built 834 rotamers at 61 positions.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating DensePD

In [27]:
#13
# Dump the pose and take a look at it.
# YOUR CODE HERE
pose.dump_pdb('2r0l_all_repack_step_13.pdb')

True

We wanted to reduce the running time. Did we? Look at the structure you just produced in pymol. Are the motions confined to the regions near H1?

----

Okay, so we know whats wrong here, right? Even though we only told the minimizer to move the H1 CDR, it moved the whole structure. Of course it did! The fold tree propagated movements down through the C-terminal regions of the pose. Print out the fold tree.
```
   print("fold tree", pose.fold_tree())
```

In [28]:
#14
# YOUR CODE HERE
print('fold tree', pose.fold_tree())

fold tree FOLD_TREE  EDGE 1 108 -1  EDGE 1 109 1  EDGE 109 225 -1  EDGE 1 226 2  EDGE 226 464 -1 


If our FoldTree confined propagations to the CDR neighborhood, then scoring and minimization would be much much faster. There are two fixes for this: modify the FoldTree or use cartesian-space refinement. We will be modifying fold trees later in this lab, and it's somewhat tricky. Lets do the easier of the two first - cartesian!

The cartesian version of the ref2015 score function includes terms for bond-stretching and bond-angle-bending; these are terms we don't need when we're working in internal geometries and are only modifying torsions. If torsions are the only things changing, then the energies of bond stretches and bond-angle bends will cancel out between two poses (wt and design) so we can just leave their energies out of the score function. When we're performing cartesian minimization, bonds stretch and bond angles bend and so we need a term to keep them in line.

(The secondary effect of this is to make minimization much more difficult, as atoms can only move so far before the bond stretch term tells them to turn around and walk back where they came from.)

In [29]:
pose.assign(original_pose)
cart_sf = create_score_function("ref2015_cart")
mmf.set_cartesian(True)

# create a new fast relax mover to make sure
# nothing from our previous run bleeds through
fr = FastRelax()
fr.set_movemap_factory(mmf)
fr.set_task_factory(pack_cdrs_and_neighbors_tf)
fr.set_scorefxn(cart_sf)
fr.cartesian(True)

#This is a general recommendation for cartesian minimization - it lowers the number of maximum cycles.
# More than this only increases time of protocol, but has little effect on energies/structure
fr.max_iter(200)

core.scoring.CartesianBondedEnergy: {0} Initializing IdealParametersDatabase with default Ks=300 , 80 , 80 , 10 , 80
basic.io.database: {0} Database file opened: scoring/score_functions/bondlength_bondangle/default-lengths.txt
core.scoring.CartesianBondedEnergy: {0} Read 759 bb-independent lengths.
basic.io.database: {0} Database file opened: scoring/score_functions/bondlength_bondangle/default-angles.txt
core.scoring.CartesianBondedEnergy: {0} Read 1434 bb-independent angles.
basic.io.database: {0} Database file opened: scoring/score_functions/bondlength_bondangle/default-torsions.txt
core.scoring.CartesianBondedEnergy: {0} Read 1 bb-independent torsions.
basic.io.database: {0} Database file opened: scoring/score_functions/bondlength_bondangle/default-improper.txt
core.scoring.CartesianBondedEnergy: {0} Read 2202 bb-independent improper tors.
core.scoring.ScoreFunctionFactory: {0} SCOREFUNCTION: ref2015


In [31]:
#14
#Run relax and measure its running time
# YOUR CODE HERE
%time fr.apply(pose)

basic.io.database: {0} Database file opened: sampling/antibodies/cluster_center_dihedrals.txt
protocols.antibody.AntibodyNumberingParser: {0} Antibody numbering scheme definitions read successfully
protocols.antibody.AntibodyNumberingParser: {0} Antibody CDR definition read successfully
antibody.AntibodyInfo: {0} Successfully finished the CDR definition
antibody.AntibodyInfo: {0} AC Detecting Regular CDR H3 Stem Type
antibody.AntibodyInfo: {0} ARFWWRSFDYW
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: KINKED
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: Kink: 1 Extended: 0
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H1
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 13 Omega: TTTTTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H2
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 10 Omega: TTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H3
protocols.antibody.cluster.CDRC

core.pose.util: {0} [ WARNING ] Unable to find atom_tree atom for this Rosetta branch connection angle: residue 422 BRANCH 1
core.pose.util: {0} [ WARNING ] Unable to find atom_tree atom for this Rosetta branch connection angle: residue 440 BRANCH 1
core.optimization.Minimizer: {0} [ WARNING ] LBFGS MAX CYCLES 200 EXCEEDED, BUT FUNC NOT CONVERGED!
protocols.relax.FastRelax: {0} CMD: min  -743.815  0.096566  0.096566  0.02805
protocols.relax.FastRelax: {0} CMD: coord_cst_weight  -743.815  0.096566  0.096566  0.02805
protocols.relax.FastRelax: {0} CMD: scale:fa_rep  -634.663  0.096566  0.096566  0.14575
core.pack.task: {0} Packer task: initialize from command line()
basic.io.database: {0} Database file opened: sampling/antibodies/cluster_center_dihedrals.txt
protocols.antibody.AntibodyNumberingParser: {0} Antibody numbering scheme definitions read successfully
protocols.antibody.AntibodyNumberingParser: {0} Antibody CDR definition read successfully
antibody.AntibodyInfo: {0} Successfully

basic.thread_manager.RosettaThreadManager: {2} Thread 2 completed 158 of 1432 work units.
basic.thread_manager.RosettaThreadManager: {7} Thread 7 completed 162 of 1432 work units.
basic.thread_manager.RosettaThreadManager: {6} Thread 6 completed 194 of 1432 work units.
basic.thread_manager.RosettaThreadManager: {3} Thread 3 completed 204 of 1432 work units.
basic.thread_manager.RosettaThreadManager: {1} Thread 1 completed 233 of 1432 work units.
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 8 available threads (8 had been requested).
protocols.relax.FastRelax: {0} CMD: repack  -533.088  0.0607842  0.0607842  0.30745
protocols.relax.FastRelax: {0} CMD: scale:fa_rep  -524.069  0.0607842  0.0607842  0.31955
core.pose.util: {0} [ WARNING ] Unable to find atom_tree atom for this Rosetta branch connection angle: residue 23 BRANCH 1
core.pose.util: {0} [ WARNING ] Unable to find atom_tree atom for this Rosetta branch connection angle: residue 88 BRANCH 

protocols.relax.FastRelax: {0} CMD: accept_to_best  -371.844  0.0415382  0.0415382  0.55
protocols.relax.FastRelax: {0} CMD: endrepeat  -371.844  0.0415382  0.0415382  0.55
protocols.relax.FastRelax: {0} CMD: coord_cst_weight  -371.844  0.0415382  0.0415382  0.55
protocols.relax.FastRelax: {0} CMD: scale:fa_rep  -731.568  0.0415382  0.0415382  0.022
core.pack.task: {0} Packer task: initialize from command line()
basic.io.database: {0} Database file opened: sampling/antibodies/cluster_center_dihedrals.txt
protocols.antibody.AntibodyNumberingParser: {0} Antibody numbering scheme definitions read successfully
protocols.antibody.AntibodyNumberingParser: {0} Antibody CDR definition read successfully
antibody.AntibodyInfo: {0} Successfully finished the CDR definition
antibody.AntibodyInfo: {0} AC Detecting Regular CDR H3 Stem Type
antibody.AntibodyInfo: {0} ARFWWRSFDYW
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: KINKED
antibody.AntibodyInfo: {0} AC Finished Det

basic.thread_manager.RosettaThreadManager: {2} Thread 2 completed 188 of 1518 work units.
basic.thread_manager.RosettaThreadManager: {0} Thread 0 completed 229 of 1518 work units.
basic.thread_manager.RosettaThreadManager: {7} Thread 7 completed 172 of 1518 work units.
basic.thread_manager.RosettaThreadManager: {3} Thread 3 completed 198 of 1518 work units.
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 8 available threads (8 had been requested).
protocols.relax.FastRelax: {0} CMD: repack  -639.643  0.0955721  0.0955721  0.14575
protocols.relax.FastRelax: {0} CMD: scale:fa_rep  -632.051  0.0955721  0.0955721  0.154
core.pose.util: {0} [ WARNING ] Unable to find atom_tree atom for this Rosetta branch connection angle: residue 23 BRANCH 1
core.pose.util: {0} [ WARNING ] Unable to find atom_tree atom for this Rosetta branch connection angle: residue 88 BRANCH 1
core.pose.util: {0} [ WARNING ] Unable to find atom_tree atom for this Rosetta branch conn

protocols.relax.FastRelax: {0} CMD: scale:fa_rep  -369.796  0.0465854  0.0465854  0.55
core.pack.task: {0} Packer task: initialize from command line()
basic.io.database: {0} Database file opened: sampling/antibodies/cluster_center_dihedrals.txt
protocols.antibody.AntibodyNumberingParser: {0} Antibody numbering scheme definitions read successfully
protocols.antibody.AntibodyNumberingParser: {0} Antibody CDR definition read successfully
antibody.AntibodyInfo: {0} Successfully finished the CDR definition
antibody.AntibodyInfo: {0} AC Detecting Regular CDR H3 Stem Type
antibody.AntibodyInfo: {0} ARFWWRSFDYW
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: KINKED
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: Kink: 1 Extended: 0
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H1
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 13 Omega: TTTTTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H2
protocols.antibo

Lets dump the pose and take a look. Did we succeed at confining backbone conformational changes to the H1 CDR? Did we make relax run faster? How are the energies? Remember to score the native with the new score function so you can make an apples-to-apples comparison.

In [33]:
#15
# YOUR CODE HERE
pose.dump_pdb('2r0l_all_repack_15.pdb')

True

Now lets do the the classic way. We need to create our FoldTree that confines the propagation of changes to the CDR so that we're not having to score so much of the Pose. After we have created the FoldTree, we will set it in our pose. Then when we run the fast relax protocol, it will use that fold tree during minimization.

To build the fold tree, we're going to take advantage of the fact that we have an Antibody here. 
The AntibodyInfo class will allow us to do build the fold tree more easily. Otherwise, we need to know the 
starting/end points of the CDR loop we are interested in. (And that's not impossible to figure out, but it takes some effort).



In [34]:
pose = original_pose.clone()

ab_info = AntibodyInfo(pose)

start = ab_info.get_CDR_start(h1, pose)
stop =  ab_info.get_CDR_end(h1, pose)
cutpoint = (stop-start)//2+start
cdr_loop = Loop(start, stop, cutpoint)
cdr_loops = Loops()
cdr_loops.add_loop(cdr_loop)
                                
ft = FoldTree()
fold_tree_from_loops(pose, cdr_loops, ft)

print("New fold tree", ft)

# for reference, hold on to the old fold tree
original_ft = pose.fold_tree()

basic.io.database: {0} Database file opened: sampling/antibodies/cluster_center_dihedrals.txt
protocols.antibody.AntibodyNumberingParser: {0} Antibody numbering scheme definitions read successfully
protocols.antibody.AntibodyNumberingParser: {0} Antibody CDR definition read successfully
antibody.AntibodyInfo: {0} Successfully finished the CDR definition
antibody.AntibodyInfo: {0} AC Detecting Regular CDR H3 Stem Type
antibody.AntibodyInfo: {0} ARFWWRSFDYW
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: KINKED
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: Kink: 1 Extended: 0
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H1
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 13 Omega: TTTTTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H2
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 10 Omega: TTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H3
protocols.antibody.cluster.CDRC

Set the new fold tree.

It is **super important** that we call the `pose.fold_tree(ft)` function
instead of modifying the pose's fold tree directly. Why? Well,
the problem is that Python doesn't respect all of the
features of C++, including "const correctness."  In
C++, it's not possible to modify the Pose's fold tree
directly: the function that grants access to the fold
tree doesn't let you modify it. It is "const". But
python doesn't understand const, and so python lets you
modify a fold tree that you shouldn't be allowed to.

Be aware that this pitfall exists! If there is a class
with a getter for some of its data and also a setter,
then when you're using that class in python, use the
setter instead of modifying the object returned by
the getter.

This particular pitfall -- modifying a Pose's fold tree directly --
has trapped many people, including senior developers.

Oh, I could name names.

In [35]:
pose.fold_tree(ft)

We need special residue types for the residues at the beginning and end of the H1 CDR. Because kinematic changes will no longer propogate across the loop boundary, bond lengths and angles can go out of whack. The "cutpoint" variants will help represent bond angles and lengths, and the chainbreak score term will make sure that the geometry stays reasonable at the loop boundaries.

In [36]:
add_cutpoint_variants(pose)

protocols.loops.loops_main: {0} Added cutpoint variant to residue 138
protocols.loops.loops_main: {0} Added cutpoint variant to residue 139


In [37]:
#Add chainbreak term so we don't get wacky stuff.  This term helps keep the peptide closed during bb movement
scorefxn_ch = scorefxn.clone()
scorefxn_ch.set_weight(rosetta.core.scoring.chainbreak, 100)

#Setup our FastRelax again for dihedral-space.
mmf.set_cartesian(False)

fr = FastRelax()
fr.set_movemap_factory(mmf)
fr.set_task_factory(pack_cdrs_and_neighbors_tf)
fr.set_scorefxn(scorefxn_ch)
fr.max_iter(0) #Default - if its 0, then we don't set max iterations it in the MinMover that FastRelax runs

core.scoring.ScoreFunctionFactory: {0} SCOREFUNCTION: ref2015


In [38]:
#16
#Run relax here
# YOUR CODE HERE
%time fr.apply(pose)

# We could reapply the original fold tree if we needed to
# pose.fold_tree(original_ft)

basic.io.database: {0} Database file opened: sampling/antibodies/cluster_center_dihedrals.txt
protocols.antibody.AntibodyNumberingParser: {0} Antibody numbering scheme definitions read successfully
protocols.antibody.AntibodyNumberingParser: {0} Antibody CDR definition read successfully
antibody.AntibodyInfo: {0} Successfully finished the CDR definition
antibody.AntibodyInfo: {0} AC Detecting Regular CDR H3 Stem Type
antibody.AntibodyInfo: {0} ARFWWRSFDYW
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: KINKED
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: Kink: 1 Extended: 0
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H1
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 13 Omega: TTTTTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H2
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 10 Omega: TTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H3
protocols.antibody.cluster.CDRC

antibody.AntibodyInfo: {0} Setting up CDR Cluster for H1
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 13 Omega: TTTTTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H2
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 10 Omega: TTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H3
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 10 Omega: TTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for L1
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 11 Omega: TTTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for L2
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 8 Omega: TTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for L3
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 9 Omega: TTTTTTCTT
core.pack.pack_rotamers: {0} built 863 rotamers at 60 positions.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_g

basic.thread_manager.RosettaThreadManager: {6} Thread 6 completed 140 of 1324 work units.
basic.thread_manager.RosettaThreadManager: {2} Thread 2 completed 207 of 1324 work units.
basic.thread_manager.RosettaThreadManager: {5} Thread 5 completed 115 of 1324 work units.
basic.thread_manager.RosettaThreadManager: {7} Thread 7 completed 114 of 1324 work units.
basic.thread_manager.RosettaThreadManager: {4} Thread 4 completed 147 of 1324 work units.
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 8 available threads (8 had been requested).
protocols.relax.FastRelax: {0} CMD: repack  -600.505  0.029709  0.029709  0.55
protocols.relax.FastRelax: {0} CMD: min  -608.47  0.02355  0.02355  0.55
protocols.relax.FastRelax: {0} MRP: 0  -608.47  -608.47  0.02355  0.02355
protocols.relax.FastRelax: {0} CMD: accept_to_best  -608.47  0.02355  0.02355  0.55
protocols.relax.FastRelax: {0} CMD: endrepeat  -608.47  0.02355  0.02355  0.55
protocols.relax.FastRelax: {0} 

protocols.antibody.AntibodyNumberingParser: {0} Antibody CDR definition read successfully
antibody.AntibodyInfo: {0} Successfully finished the CDR definition
antibody.AntibodyInfo: {0} AC Detecting Regular CDR H3 Stem Type
antibody.AntibodyInfo: {0} ARFWWRSFDYW
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: KINKED
antibody.AntibodyInfo: {0} AC Finished Detecting Regular CDR H3 Stem Type: Kink: 1 Extended: 0
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H1
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 13 Omega: TTTTTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H2
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 10 Omega: TTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for H3
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 10 Omega: TTTTTTTTTT
antibody.AntibodyInfo: {0} Setting up CDR Cluster for L1
protocols.antibody.cluster.CDRClusterMatcher: {0} Length: 11 Omega: TTTTTTTTTTT
antibody.Anti

How does the pose look?  Better/worse than cartesian? No difference? How do the energies compare? Did it run faster than cartesian?

In [39]:
#17
# YOUR CODE HERE
#Lets compare the energies of the before and after packing. Any difference?
scorefxn = get_score_function()
before = scorefxn.score(original_pose)
#Finish the code here
# YOUR CODE HERE
after = scorefxn.score(pose)
print('Difference in energies =', after - before)

core.scoring.ScoreFunctionFactory: {0} SCOREFUNCTION: ref2015
Difference in energies = -87.78017293815071


That should get you started with packing/relax! We even covered some advanced topics you will see later this week such as the TaskFactory and Residue Selectors! If you have any questions, please ask!

A final thing to consider.

We made relax faster by changing less of the structure. But because less of the structure moved, we also made it more accurate. FastRelax is not a deterministic mover. If you invoke it 10 times, you will get 10 different answers. The larger the structure you're modeling, the more noise you will introduce. Is one H1 conformation better than another because of the contacts it forms at the interface, or is its lower energy due to FastRelax relieving strain on the other side of the molecule that it couldn't in the other invocations? The more DOFs you open up, the more sampling you have to do to overcome the noise.